# Homework3 Dynamic Programming
¶

Adapt the editDistance function we saw in practical (copied below) to answer questions 1 and 2 below. Your function should take arguments p (pattern), t (text) and should return the edit distance of the match between P and T with the fewest edits.

In [1]:
def editDistance(x, y):
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = i
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return D[-1][-1]

In [18]:
def editDistanceApproximate(P, T):
    m, n = len(P), len(T)
    D = [[0 for j in range(n+1)] for i in range(m+1)]
    for i in range(m+1): 
      D[i][0] = i # init first column by distance from empty string
        
#   the first row is all 0s unlike edit distance, since there is no bias toward alignment
#   of P in T from the beginning of both P and T, (ie. P can start at any index in T)

#
#   for j in range(n+1): dp[0][j] = j # DELETED!!!
#

    for i in range(1, m+1):
        for j in range(1, n+1):
            D[i][j] = min(
                D[i-1][j-1] + int(P[i-1] != T[j-1]),
                D[i-1][j] + 1,
                D[i][j-1] + 1,
            )
    return min(D[m])

In [2]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [4]:
!wget https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta

--2022-03-04 00:34:03--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 13.32.84.231, 13.32.84.169, 13.32.84.71, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|13.32.84.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810105 (791K) [application/octet-stream]
Saving to: ‘chr1.GRCh38.excerpt.fasta’

chr1.GRCh38.excerpt 100%[===================>] 791.12K  --.-KB/s    in 0.1s    

2022-03-04 00:34:04 (6.01 MB/s) - ‘chr1.GRCh38.excerpt.fasta’ saved [810105/810105]



In [19]:
%%time
genome = 'chr1.GRCh38.excerpt.fasta'

t = readGenome(genome)

# Question1
p = 'GCTGATCGATCGTACG'
print(editDistanceApproximate(p, t))

3
CPU times: user 12.9 s, sys: 63.3 ms, total: 13 s
Wall time: 13 s


In [20]:
%%time
# Question2
p = 'GATTTACCAGATTGAG'
print(editDistanceApproximate(p, t))

2
CPU times: user 12.8 s, sys: 36.7 ms, total: 12.8 s
Wall time: 12.8 s


In [7]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities


In [8]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

In [9]:
def smart_overlap_map(reads, k):
    olaps = {}
    result = {}
    for read in reads:
        for i in range(len(read)-k+1):
            if read[i:i+k] not in olaps:
                olaps[read[i:i+k]] = [read]
            else:
                olaps[read[i:i+k]].append(read)

    count = 0
    for read in reads:
        read_suffix = read[-k:]
        for possible_read in olaps[read_suffix]:
            if possible_read != read:
                olen = overlap(read, possible_read, k)
                if olen > 0:
                    count += 1
                    result[(read, possible_read)] = olen

    return result, count

In [13]:
!wget https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq

--2022-03-04 00:46:42--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 13.32.84.71, 13.32.84.31, 13.32.84.169, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|13.32.84.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2562951 (2.4M) [application/octet-stream]
Saving to: ‘ERR266411_1.for_asm.fastq’

ERR266411_1.for_asm 100%[===================>]   2.44M  13.3MB/s    in 0.2s    

2022-03-04 00:46:42 (13.3 MB/s) - ‘ERR266411_1.for_asm.fastq’ saved [2562951/2562951]



In [16]:
reads_file = 'ERR266411_1.for_asm.fastq'
reads, _ = readFastq(reads_file)
pairs_olen, pairs_count = smart_overlap_map(reads, 30)

# Question 3
print("Number of pairs of reads overlap:", pairs_count)

Number of pairs of reads overlap: 904746


In [15]:
# Question 4
reads_involved = []
for key, value in pairs_olen:
    reads_involved.append(key)
print("Number of reads have a suffix involved in an overlap:", len(set(reads_involved)))

Number of reads have a suffix involved in an overlap: 7161
